This file is to practice and build familiratity with Loading data into Local Postgressql database post extraction serving as the precursor of the to redshift.

In [5]:
print('hello world')

hello world


In [6]:
import psycopg2 
import os
from dotenv import load_dotenv
import datetime
import calendar

In [7]:
DATABASE= os.environ.get("postgres_db")
HOST=os.environ.get("postgres_host")
USER=os.environ.get("postgres_user")
PASSWORD=os.environ.get("postgres_pass")

load_dotenv()

def setup_db_connection(database=DATABASE,host= HOST,user= USER,password= PASSWORD):
    connection = psycopg2.connect(database=database,host=host,user=user,password=password)
    cursor = connection.cursor()
    print("We connected!")
    return connection, cursor

In [8]:
connection, cursor = setup_db_connection()

We connected!


In [9]:
import pandas as pd 
import uuid
import datetime
import calendar
from etl_packages import *

In [10]:
billing_df = pd.read_csv('billing_pricing_dataset.csv')
complaints_df = pd.read_csv('complaints_dataset.csv')
customer_feedback_df = pd.read_csv('customer_feedback_dataset.csv')
provider_performance_df = pd.read_csv('provider_performance_dataset.csv')
reg_compliance_df = pd.read_csv('regulatory_compliance_dataset.csv')
service_outages_df = pd.read_csv('service_outages_dataset.csv')

In [11]:
billing_df = billing_df.dropna()
complaints_df = complaints_df.dropna()
customer_feedback_df = customer_feedback_df.dropna()
provider_performance_df = provider_performance_df.dropna()
service_outages_df = service_outages_df.dropna()
reg_compliance_df = reg_compliance_df.dropna()

In [12]:
complaints_df = complaints_df.rename(columns={'Date':'date'})
customer_feedback_df = customer_feedback_df.rename(columns={'Date':'date'})
service_outages_df = service_outages_df.rename(columns={'Date':'date'})
reg_compliance_df = reg_compliance_df.rename(columns={'AuditDate':'date'})

In [13]:
date_table_database = pd.read_sql_query("SELECT * FROM date_table", connection)
date_table_database

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_3620\3193190986.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  date_table_database = pd.read_sql_query("SELECT * FROM date_table", connection)


,dateid,dateday,dateweekday,datemonth,dateyear,date
0,06c89aae-b4fe-4545-a369-4756bc5ab23d,19,Friday,November,2021,2021-11-19
1,3f71402a-055f-47a9-92f5-141d8de7bda9,10,Sunday,September,2023,2023-09-10
2,bd4805ef-916f-41ce-b6ca-794eee5bd09c,16,Thursday,February,2023,2023-02-16
3,147d062f-2bb1-4ef1-9b10-25fc76117f2c,08,Tuesday,March,2022,2022-03-08
4,678eb070-9ddc-4b15-8ec0-b8f12d4d3dc6,04,Saturday,June,2022,2022-06-04
...,...,...,...,...,...,...
19001,4e3ba9f4-2f7d-4d03-a980-9ada375fb515,06,Thursday,April,2023,2023-04-06
19002,9ca64139-9beb-4c0c-b5fc-97933f7a0643,13,Saturday,November,2021,2021-11-13
19003,e012c7f1-f775-4a56-a938-ea667661cd62,21,Tuesday,March,2023,2023-03-21
19004,294b9d72-bd5e-4b21-8d9e-fecb80d86afa,05,Saturday,March,2022,2022-03-05


In [14]:
date_table = pd.DataFrame(columns=['dateid', 'dateday', 'dateweekday', 'datemonth', 'dateyear', 'date'])
date_table

,dateid,dateday,dateweekday,datemonth,dateyear,date


In [15]:
date_table = date_extractor(complaints_df,date_table)
date_table

,dateid,dateday,dateweekday,datemonth,dateyear,date
0,5da22022-7e84-4eff-86cd-04b6150aa9ab,19,Friday,November,2021,2021-11-19
1,6301d8ab-bd2e-4785-b2f2-386a2f6a64d4,10,Sunday,September,2023,2023-09-10
2,39c411c9-fcfa-4f58-8821-22be2bc3ae5b,16,Thursday,February,2023,2023-02-16
3,22cfdd36-9e74-47fd-aad3-46b8054c27b3,08,Tuesday,March,2022,2022-03-08
4,d44c2a1c-ea84-4859-8c7f-dfbec4f2025f,04,Saturday,June,2022,2022-06-04
...,...,...,...,...,...,...
726,108ad4c5-c5d5-4c95-8bea-e916bc99d027,06,Thursday,April,2023,2023-04-06
727,86061906-8139-407e-87dd-f1c8bc4ef33a,13,Saturday,November,2021,2021-11-13
728,3365e3c4-40c0-480e-bee2-fff830940860,21,Tuesday,March,2023,2023-03-21
729,8105c045-44ec-4baf-8866-9a7985b7ff37,05,Saturday,March,2022,2022-03-05


In [16]:

date_table = date_extractor(complaints_df,date_table)
date_table = date_extractor(customer_feedback_df,date_table)
date_table = date_extractor(service_outages_df,date_table)
date_table = date_extractor(reg_compliance_df,date_table)

In [17]:
def duplicate_drop(new_table, database_table, field):
    """
    Removes duplicates from `new_table` based on `field` matching rows in `database_table`.

    Parameters:
        new_table (pd.DataFrame): The DataFrame to clean.
        database_table (pd.DataFrame): The DataFrame to check for duplicates.
        field (str): The column name to compare.

    Returns:
        pd.DataFrame: The cleaned DataFrame with duplicates removed.
    """
    # Find duplicates using a vectorized operation
    duplicates = new_table[field].isin(database_table[field])
    # duplicates essentially creates a yes or check list for entry, where if there is duplicate 

    # Count duplicates
    counter = duplicates.sum()
    # Drop duplicates
    new_table = new_table[~duplicates]
    # ~ this here  flips the true into false and false into true. 

    print(f'The number of duplicate rows found and dropped is {counter}')
    return new_table
#instead of just making this for just one function I want to utlise it again and again for other functions

In [18]:
duplicate_drop(date_table,date_table_database,'date')

The number of duplicate rows found and dropped is 731


,dateid,dateday,dateweekday,datemonth,dateyear,date


In [19]:
import pandas as pd

df1 = pd.DataFrame({'date': ['2023-11-21', '2023-11-23', '2023-11-22', '2023-11-24', '2023-11-25']})

df2 = pd.DataFrame({'date': ['2023-11-22', '2023-11-24']})

In [20]:
df1 = duplicate_drop(df1,df2,'date')

The number of duplicate rows found and dropped is 2


In [21]:
# Here we have the for loop to load the data into a database

# sql = """
#         INSERT INTO date_table (dateid, dateday,dateweekday,datemonth,dateyear,date) 
#         VALUES (%s, %s, %s, %s, %s, %s)
#         """
# for index, row in date_table.iterrows():
#     data_values = (row['dateid'],row['dateday'],row['dateweekday'],
#                    row['datemonth'],row['dateyear'],row['date'])
#     cursor.execute(sql, data_values)
#     connection.commit()

In [22]:
def load_date_into_table():
    sql = """
        INSERT INTO date_table (dateid, dateday,dateweekday,datemonth,dateyear,date) 
        VALUES (%s, %s, %s, %s, %s, %s)
        """
    for index, row in date_table.iterrows():
        data_values = (row['dateid'],row['dateday'],row['dateweekday'],
                   row['datemonth'],row['dateyear'],row['date'])
        cursor.execute(sql, data_values)
        connection.commit()

In [23]:
load_date_into_table()

In [24]:
customer_table_database = pd.read_sql_query("SELECT * FROM customer_table", connection)
customer_table_database

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_3620\1111470028.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_table_database = pd.read_sql_query("SELECT * FROM customer_table", connection)


,customer_id,customer_name,customer_phone_number
0,fa4609b1-def1-41a0-86fc-93c59a9b2a91,Danielle Marquez,23481676113974
1,f6257f2b-a674-41f7-80a3-4293b72df1fe,Jeffrey Schaefer,23490849550529
2,0fb12280-d4f8-4ac5-8815-a2b17f40d9e8,Lindsey Guzman,23481565521241
3,b9788848-1d9c-4ff2-80b1-c129b887c22e,Kimberly Garcia,23490886905937
4,8c850ebe-3e18-401d-9f6b-7fb0f1203b5f,Alicia Jones,2347042661204
...,...,...,...
17899,fd40dfa9-0085-436a-a7b8-8c9e98bd8156,Caitlyn Ayers,23490855095191
17900,8527e94a-4a05-4c22-94a3-5e1a01b3a8d8,Joseph Gonzalez,23480393446241
17901,5e4c5527-e1ea-43b6-b6ab-ab305146999e,Linda Hamilton,23470723971114
17902,9598e9c4-2838-45b1-88fb-c07c2a91c0e3,Deborah Nelson,23481723607310


In [25]:
customer_table = pd.DataFrame(columns=['customer_id', 'customer_name', 'customer_phone_number'])
customer_table

,customer_id,customer_name,customer_phone_number


In [26]:
customer_table = customer_name_extractor(customer_feedback_df,customer_table)

In [27]:
customer_table = customer_name_extractor(complaints_df,customer_table)

In [28]:
customer_table = duplicate_drop(customer_table, customer_table_database,'customer_name')

The number of duplicate rows found and dropped is 17904


In [29]:
customer_table

,customer_id,customer_name,customer_phone_number


In [30]:
def load_customer_data_into_table():
    sql = """
        INSERT INTO customer_table (customer_id, customer_name, customer_phone_number) 
        VALUES (%s, %s, %s)
        """
    for index, row in customer_table.iterrows():
        customer_values = (row['customer_id'],row['customer_name'],row['customer_phone_number'])
        cursor.execute(sql, customer_values)
        connection.commit()

In [31]:
load_customer_data_into_table()

I Will have to add a if loop so that if there is nothing to load nothing is loaded 

######Provider_Table####

In [32]:
provider_table_database = pd.read_sql_query("SELECT * FROM provider_table", connection)
provider_table_database

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_3620\1136414376.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  provider_table_database = pd.read_sql_query("SELECT * FROM provider_table", connection)


,providerid,providername
0,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6,Glo
1,5a484229-b5ec-473b-a289-370f02c0d6ad,Airtel
2,a53a2fb3-bf32-4f6a-b1f2-f4e03217e1f5,Etisalat
3,a787c2c7-d1ba-41a7-adca-e10fccda98c6,MTN


In [33]:
provider_table = pd.DataFrame(columns=['providerid', 'providername'])


In [34]:

provider_table

provider_table = provider_name_extractor(provider_performance_df,provider_table)


In [35]:
provider_table = provider_name_extractor(reg_compliance_df,provider_table)

In [36]:
# check for duplicate 
def load_provider_data_into_table():
    sql = """
        INSERT INTO provider_table (providerid, providername) 
        VALUES (%s, %s)
        """
    for index, row in provider_table.iterrows():
        provider_values = (row['providerid'],row['providername'])
        cursor.execute(sql, provider_values)
        connection.commit()


In [37]:
provider_table = duplicate_drop(provider_table,provider_table_database,'providername')

The number of duplicate rows found and dropped is 4


In [38]:
load_provider_data_into_table()

In [39]:
billing_table = ''
billing_table = billing_extractor(billing_df,billing_table,provider_table_database)

In [40]:
billing_table

,billingid,pricingplan,tariffchanges,billingdisputes,providerid
0,f4a60d3e-bf3f-48a6-a29b-0d191a80ec51,Design thus participant summer treatment stock.,Vote little blood general.,Measure born million.,5a484229-b5ec-473b-a289-370f02c0d6ad
1,527f52b8-737c-413e-8ae3-de026b141deb,Central maintain light style rate many recent.,Charge become create western once drive owner.,Everyone water clearly sing.,5a484229-b5ec-473b-a289-370f02c0d6ad
2,5c0056cd-d95f-47b2-b55a-e7bf26be36e2,Benefit sure baby anyone site perhaps generation.,Career find color job some.,Management finally offer now west before.,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6
3,ec2fcd38-5924-421d-a7be-793668220cd7,Hit rate scientist system.,Leg their door.,I style company away include finally catch.,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6
4,d685bef3-4448-4189-a4cb-1d5f58654e27,Spring well coach conference.,Use say total employee step thing century cand...,Money near beyond.,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6
...,...,...,...,...,...
9995,1c7cdbb0-dabf-47d2-9e4e-37c3fca8bf48,On though at building sound southern.,Raise safe assume or admit.,State job population.,a787c2c7-d1ba-41a7-adca-e10fccda98c6
9996,3e13e834-6bfa-488c-b6c7-c4cc0bad50a6,Debate game area course wish feel open.,Office dark skill measure.,Chair finally big chair store.,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6
9997,ce9a21e1-bea9-4d07-919e-b8f454a762d3,Bit send attorney something sort.,Them poor role opportunity system traditional ...,Individual offer go when son simple industry s...,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6
9998,588f3566-c008-4ec0-91da-176bc45db98c,Majority Democrat take financial yard about po...,Southern second true off fall share also remem...,See commercial piece visit.,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6


In [41]:
def load_billing_data_into_table():
    sql = """
        INSERT INTO billing_table (billingid, pricingplan, tariffchanges, billingdisputes, providerid) 
        VALUES (%s, %s, %s, %s, %s)
        """
    for index, row in billing_table.iterrows():
        billing_values = (row['billingid'],row['pricingplan'],row['tariffchanges'],row['billingdisputes'],row['providerid'])
        cursor.execute(sql, billing_values)
        connection.commit()


In [42]:
# load_billing_data_into_table()

In [43]:
reg_compliance_df

,ProviderID,ProviderName,ComplianceStatus,RegulatoryRequirements,date
0,92,Glo,Compliant,Take notice subject even bag before beat.,2023-04-30
1,97,Etisalat,Compliant,Book tough charge operation.,2023-06-08
2,38,Airtel,Non-Compliant,Direction particular prepare at.,2022-11-17
3,14,Etisalat,Compliant,Language poor technology money company week.,2022-08-02
4,72,Etisalat,Non-Compliant,Stock trouble them ok tonight.,2022-07-15
...,...,...,...,...,...
9995,57,Etisalat,Non-Compliant,Not nation plant other.,2023-03-22
9996,22,MTN,Non-Compliant,Everything help beyond lead.,2022-08-10
9997,80,Glo,Non-Compliant,Gun less book in.,2022-11-23
9998,49,Airtel,Non-Compliant,Through become rate body real me.,2023-03-04


In [44]:
regulatory_compliance_table = ''
regulatory_compliance_table = regulatory_compliance_extractor(reg_compliance_df,regulatory_compliance_table,provider_table_database,date_table_database)

In [45]:
regulatory_compliance_table 

,complianceid,regulatoryrequirements,compliancestatuss,dateid,providerid
0,1e31d138-e4ef-4bef-b92b-d0b9753bc5d1,Take notice subject even bag before beat.,Compliant,c1a3a9e0-4ea0-4f7a-b1cf-1878457ccd1c,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6
1,444faaae-a25c-40e2-b67b-86618b32fdcb,Book tough charge operation.,Compliant,14b95f75-63ea-4a80-adcf-661b3759a000,a53a2fb3-bf32-4f6a-b1f2-f4e03217e1f5
2,8ea6e8ec-ef2a-438c-8b0e-d4628eaa35fa,Direction particular prepare at.,Non-Compliant,d04ba6c0-b12b-414e-b142-98aea3658892,5a484229-b5ec-473b-a289-370f02c0d6ad
3,672e680a-1c40-42d4-98ea-057e0e7a2439,Language poor technology money company week.,Compliant,0b66e8a3-a499-405e-ac4d-e9f662ceee4c,a53a2fb3-bf32-4f6a-b1f2-f4e03217e1f5
4,9a868987-1066-4d42-acea-13f0fd907d03,Stock trouble them ok tonight.,Non-Compliant,dbc485e8-6408-4587-89f6-b3e65276a5bc,a53a2fb3-bf32-4f6a-b1f2-f4e03217e1f5
...,...,...,...,...,...
9995,81676395-dd73-461f-962e-a1a893e15438,Not nation plant other.,Non-Compliant,3f39ba11-9eab-4a8e-9c03-668cfaceb630,a53a2fb3-bf32-4f6a-b1f2-f4e03217e1f5
9996,2fa03029-e360-4a25-91bb-03082eac55f2,Everything help beyond lead.,Non-Compliant,9c6cb828-1d8f-42d2-a2eb-385c083b7c47,a787c2c7-d1ba-41a7-adca-e10fccda98c6
9997,9d355600-e25c-4c68-b210-06a50a93e017,Gun less book in.,Non-Compliant,ba2a66b0-7b14-4f20-9066-b8ed300a5e73,e16e7e07-878e-48aa-a70f-a5e8d7fb66a6
9998,1888262f-c191-4bd2-82a7-2ceea2a04f9c,Through become rate body real me.,Non-Compliant,608270f8-e4f1-4fba-8761-5f504bf33c8c,5a484229-b5ec-473b-a289-370f02c0d6ad


In [46]:
def load_compliance_data_into_table():
    sql = """
        INSERT INTO regulatory_compliance_table (complianceid, regulatoryrequirements, compliancestatuss, dateid, providerid) 
        VALUES (%s, %s, %s, %s, %s)
        """
    for index, row in regulatory_compliance_table.iterrows():
        comp_values = (row['complianceid'],row['regulatoryrequirements'],row['compliancestatuss'],row['dateid'],row['providerid'])
        cursor.execute(sql, comp_values)
        connection.commit()				

In [47]:
# load_compliance_data_into_table()

In [48]:
service_outages_df

,OutageID,date,ServiceType,ServiceProvider,OutageType,Duration,AffectedAreas,Description
0,4423,2021-10-17,Pay TV,Airtel,Service,17.104040,Thomashaven,Ability police happen together.
1,7841,2022-06-30,Pay TV,Etisalat,Downtime,16.719199,Georgeside,Staff bar much.
2,3747,2022-10-28,Pay TV,Etisalat,Downtime,7.912462,Crystalberg,Really past attention low system respond.
3,2798,2023-09-14,Pay TV,Airtel,Downtime,21.309589,Cassandraville,Audience tell information four Democrat choice...
4,6047,2022-10-02,Pay TV,MTN,Service,13.994673,Reginaldside,Class surface our community on state.
...,...,...,...,...,...,...,...,...
9995,5765,2021-11-10,Pay TV,Airtel,Network,14.321467,Ashleytown,Kid relationship behind right computer PM whet...
9996,2472,2021-11-12,Telecom,Etisalat,Downtime,2.172465,North Michael,Modern safe significant western assume differe...
9997,1714,2022-06-17,Pay TV,MTN,Service,14.778416,West Michael,Anyone home lay box indeed.
9998,4178,2022-04-17,Telecom,Airtel,Downtime,5.036795,Higginsstad,Federal report good assume control would pull.


In [49]:
from etl_packages import *

In [50]:
service_outages_table =''
service_outages_table = service_outage_extractor(service_outages_df,service_outages_table,date_table_database)

In [51]:
service_outages_table

,outageid,outagetype,duration,affectedAreas,description,dateid
0,045514f6-3983-4054-b15b-6f144881af21,Service,17.104040,Thomashaven,Ability police happen together.,2172a032-1869-4623-aa49-084e59067b25
1,d808ba57-698b-4edd-b3ea-7cff151618fa,Downtime,16.719199,Georgeside,Staff bar much.,0babd716-fdd8-4ece-a6d6-bb30d0a98ce7
2,87a8f0df-6668-4569-851f-dc3ae64b479f,Downtime,7.912462,Crystalberg,Really past attention low system respond.,4614f4e1-23ac-43e7-a555-5f50bd4d81a3
3,cbd84ce4-b337-4a11-9c7d-ca69585bff05,Downtime,21.309589,Cassandraville,Audience tell information four Democrat choice...,99cadf5b-65ed-488a-8e66-618efe5eb716
4,f2fe337c-f0d0-406c-b085-f095c9d1775a,Service,13.994673,Reginaldside,Class surface our community on state.,03f7da84-d8fe-4189-bf6c-a9843674ef6d
...,...,...,...,...,...,...
9995,9b3cd134-c26b-4867-9847-e17fb8b984b4,Network,14.321467,Ashleytown,Kid relationship behind right computer PM whet...,d5df0df7-4e62-4b99-ad59-6683e76676ac
9996,2988d8fb-cf2f-4866-b7ad-b107526f0108,Downtime,2.172465,North Michael,Modern safe significant western assume differe...,a6963744-d1fd-4ce3-bbf7-4556895eb0e2
9997,39e0b66e-e382-42bd-822a-a062f39e1b3a,Service,14.778416,West Michael,Anyone home lay box indeed.,6abc1b6d-8356-4ed0-b7af-df381e4e0c67
9998,e551fb0a-07fc-4882-94d0-0f5336c186e7,Downtime,5.036795,Higginsstad,Federal report good assume control would pull.,ef4708b7-750e-408f-93ec-f43c588794d1


In [52]:
			
def load_service_outages_data_into_table():
    sql = """
        INSERT INTO service_outages_table (outageid, outagetype, duration, affectedAreas, description, dateid) 
        VALUES (%s, %s, %s, %s, %s, %s)
        """
    for index, row in service_outages_table.iterrows():
        outage_values = (row['outageid'],row['outagetype'],row['duration'],row['affectedAreas'],row['description'],row['dateid'])
        cursor.execute(sql, outage_values)
        connection.commit()	

In [53]:
load_service_outages_data_into_table()